In [1]:
import os

In [ ]:

os.chdir("../")


In [4]:
%pwd

'c:\\Users\\pc\\Desktop\\ai\\mlops-datascience'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataIngestionconfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [ ]:
from src.mlops_pipeline.constants import *
from src.mlops_pipeline.utils.common import read_yaml,create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema= read_yaml(schema_filepath)
        
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionconfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionconfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir,
        )
        
        return data_ingestion_config

In [ ]:
import urllib.request as request
from src.mlops_pipeline import logger

In [ ]:
#DataIngestion Component

class DataIngestion:
    def __init__(self,config:DataIngestionconfig):
        self.config=config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info \n{headers}")
        else:
            logger.info(f"Rile already exists")
        